In [ ]:
import pandas as pd
import requests
import time

# ✅ 카카오 REST API 키 입력
KAKAO_API_KEY = 'c3ac59b5fd5e4adc95549f12546820b7'

# ✅ 주소 → 좌표 변환 함수
def get_coordinates(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        #200은 성공했다라는 코드값
        return None, None

    result = response.json()
    documents = result.get("documents", [])
    if documents:
        x = float(documents[0]["x"])  # 경도
        y = float(documents[0]["y"])  # 위도
        return y, x
    return None, None

# ✅ 주소 목록 CSV 파일 불러오기
df = pd.read_csv("input.csv")  # 주소가 들어 있는 열 이름은 '주소'라고 가정

# ✅ 위도, 경도 컬럼 추가
lat_list = []
lon_list = []

for address in df['주소']:
    lat, lon = get_coordinates(address)
    lat_list.append(lat)
    lon_list.append(lon)
    time.sleep(0.2)  # 과도한 요청 방지를 위해 0.2초 지연

df['위도'] = lat_list
df['경도'] = lon_list

# ✅ 결과 저장
df.to_csv("기업주소_위경도_결과.csv", index=False, encoding='utf-8-sig')
print("완료: 기업주소_위경도_결과.csv 저장됨")


In [ ]:
import pandas as pd
import folium

# ✅ CSV 불러오기
df = pd.read_csv("기업주소_위경도_결과.csv")

# ✅ 지도 초기화 (첫 번째 좌표 기준)
map_center = [df['위도'][0], df['경도'][0]]
m = folium.Map(location=map_center, zoom_start=12)

# ✅ 인덱스를 마커로 표시
for idx, row in df.iterrows():
    lat = row['위도']
    lon = row['경도']
    
    # 유효한 좌표만 마킹
    if pd.notna(lat) and pd.notna(lon):
        folium.Marker(
            [lat, lon],
            popup=f"Index: {idx}",
            tooltip=row['주소']
        ).add_to(m)

# ✅ 지도 저장
#m.save("index_marked_map.html")
#print("지도 저장 완료: index_marked_map.html")
